## 동일한 OpenSearch Serverless Collection에 여러 Knowledge Base 생성하기

이 노트북에서는 하나의 OpenSearch Serverless Collection을 공유하면서 여러 개의 Knowledge Base를 생성하는 방법을 시연합니다.

각 Knowledge Base는:
- 동일한 OSS Collection 사용
- 서로 다른 벡터 인덱스 사용
- 독립적인 Data Source 관리

#### 장점:
- OSS Collection 비용 절감 (하나만 생성)
- 각 KB는 독립적으로 관리 가능
- 서로 다른 청킹 전략 적용 가능

## 1. 필요한 라이브러리 불러오기

In [1]:
%pip install --upgrade pip --quiet
%pip install -r ../requirements.txt --no-deps --quiet
%pip install -r ../requirements.txt --upgrade --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
import os
import sys
import time
import boto3
import logging
import pprint
import json

from pathlib import Path
current_path = Path().resolve()
current_path = current_path.parent
if str(current_path) not in sys.path:
    sys.path.append(str(current_path))

from utils.knowledge_base import BedrockKnowledgeBase

In [ ]:
# Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

In [ ]:
# 타임스탬프 생성
current_time = time.time()
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
suffix = f"{timestamp_str}"

# 공유할 OSS Collection 이름
shared_collection_name = f'shared-rag-collection-{suffix}'

# 각 KB 설정
kb_configs = [
    {
        "name": f"kb-fixed-chunking-{suffix}",
        "description": "Knowledge Base with Fixed Size Chunking",
        "chunking_strategy": "FIXED_SIZE",
        "bucket_name": f"kb-fixed-{suffix}"
    },
    {
        "name": f"kb-hierarchical-chunking-{suffix}",
        "description": "Knowledge Base with Hierarchical Chunking",
        "chunking_strategy": "HIERARCHICAL",
        "bucket_name": f"kb-hierarchical-{suffix}"
    },
    {
        "name": f"kb-semantic-chunking-{suffix}",
        "description": "Knowledge Base with Semantic Chunking",
        "chunking_strategy": "SEMANTIC",
        "bucket_name": f"kb-semantic-{suffix}"
    }
]

foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"

print(f"Shared Collection Name: {shared_collection_name}")
print(f"Number of KBs to create: {len(kb_configs)}")

## 2. 첫 번째 Knowledge Base 생성 (OSS Collection 포함)

첫 번째 KB를 생성하면서 OpenSearch Serverless Collection도 함께 생성합니다.

In [ ]:
# 첫 번째 KB 설정
first_kb_config = kb_configs[0]
data_source_1 = [{"type": "S3", "bucket_name": first_kb_config["bucket_name"]}]

print(f"Creating first KB: {first_kb_config['name']}")
print(f"This will also create the shared OSS Collection: {shared_collection_name}")

# 첫 번째 KB 생성 (OSS Collection도 함께 생성됨)
kb_1 = BedrockKnowledgeBase(
    kb_name=first_kb_config["name"],
    kb_description=first_kb_config["description"],
    data_sources=data_source_1,
    chunking_strategy=first_kb_config["chunking_strategy"],
    suffix=f"{suffix}-1"
)

# OSS Collection 정보 저장
collection_id = kb_1.collection_id
collection_arn = kb_1.collection_arn
oss_host = kb_1.host

print(f"\n✓ First KB created successfully")
print(f"Collection ID: {collection_id}")
print(f"Collection ARN: {collection_arn}")
print(f"OSS Host: {oss_host}")

In [ ]:
# 버킷 생성 함수
def create_bucket_if_not_exists(bucket_name):
    """S3 버킷이 존재하지 않으면 생성합니다."""
    try:
        s3_client.head_bucket(Bucket=bucket_name)
        print(f"Bucket {bucket_name} already exists")
    except:
        print(f"Creating bucket {bucket_name}...")
        try:
            if region == 'us-east-1':
                s3_client.create_bucket(Bucket=bucket_name)
            else:
                s3_client.create_bucket(
                    Bucket=bucket_name,
                    CreateBucketConfiguration={'LocationConstraint': region}
                )
            print(f"✓ Bucket {bucket_name} created successfully")
        except Exception as e:
            print(f"Error creating bucket: {e}")
            raise

# 데이터 업로드 함수
def upload_directory(path, bucket_name):
    # 버킷이 없으면 생성
    create_bucket_if_not_exists(bucket_name)
    
    for root, dirs, files in os.walk(path):
        for file in files:
            file_to_upload = os.path.join(root, file)
            if file not in ["LICENSE", "NOTICE", "README.md"]:
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload, bucket_name, file)
            else:
                print(f"Skipping file {file_to_upload}")

# 첫 번째 KB에 데이터 업로드
upload_directory("../synthetic_dataset", first_kb_config["bucket_name"])

In [ ]:
# Ingestion 작업 시작
time.sleep(30)
kb_1.start_ingestion_job()
kb_1_id = kb_1.get_knowledge_base_id()
print(f"KB 1 ID: {kb_1_id}")

## 3. 기존 OSS Collection을 사용하여 추가 Knowledge Base 생성

이제 이미 생성된 OSS Collection을 재사용하여 추가 KB들을 생성합니다.

**핵심:** BedrockKnowledgeBase 클래스는 이미 존재하는 Collection을 감지하면 새로 생성하지 않고 재사용합니다.

In [ ]:
# 나머지 KB들을 생성 (동일한 OSS Collection 사용)
created_kbs = [kb_1]  # 첫 번째 KB 추가
kb_ids = [kb_1_id]

for i, kb_config in enumerate(kb_configs[1:], start=2):
    print(f"\n{'='*80}")
    print(f"Creating KB {i}: {kb_config['name']}")
    print(f"Chunking Strategy: {kb_config['chunking_strategy']}")
    print(f"{'='*80}")
    
    data_source = [{"type": "S3", "bucket_name": kb_config["bucket_name"]}]
    
    # KB 생성 (기존 OSS Collection 재사용)
    kb = BedrockKnowledgeBase(
        kb_name=kb_config["name"],
        kb_description=kb_config["description"],
        data_sources=data_source,
        chunking_strategy=kb_config["chunking_strategy"],
        suffix=f"{suffix}-{i}"
    )
    
    # 데이터 업로드
    upload_directory("../synthetic_dataset", kb_config["bucket_name"])
    
    # Ingestion 작업
    time.sleep(30)
    kb.start_ingestion_job()
    
    kb_id = kb.get_knowledge_base_id()
    created_kbs.append(kb)
    kb_ids.append(kb_id)
    
    print(f"\n✓ KB {i} created successfully")
    print(f"KB ID: {kb_id}")
    print(f"Index Name: {kb.index_name}")
    print(f"Using Collection ID: {kb.collection_id}")

print(f"\n{'='*80}")
print(f"All {len(created_kbs)} Knowledge Bases created successfully!")
print(f"Shared OSS Collection: {collection_id}")
print(f"{'='*80}")

## 4. 생성된 Knowledge Base 정보 확인

In [ ]:
# 모든 KB 정보 출력
print("\n" + "="*80)
print("Created Knowledge Bases Summary")
print("="*80)

for i, (kb, kb_id, config) in enumerate(zip(created_kbs, kb_ids, kb_configs), start=1):
    print(f"\nKB {i}:")
    print(f"  Name: {config['name']}")
    print(f"  ID: {kb_id}")
    print(f"  Chunking Strategy: {config['chunking_strategy']}")
    print(f"  Index Name: {kb.index_name}")
    print(f"  S3 Bucket: {config['bucket_name']}")

print(f"\nShared Resources:")
print(f"  OSS Collection ID: {collection_id}")
print(f"  OSS Collection ARN: {collection_arn}")
print(f"  OSS Host: {oss_host}")
print("="*80)

## 5. 각 Knowledge Base 테스트

동일한 질의로 각 KB를 테스트하여 청킹 전략에 따른 차이를 확인합니다.

In [ ]:
query = "Provide a summary of consolidated statements of cash flows of Octank Financial for the fiscal years ended December 31, 2019."

print(f"Query: {query}\n")
print("="*80)

In [ ]:
# 각 KB에 대해 테스트
for i, (kb_id, config) in enumerate(zip(kb_ids, kb_configs), start=1):
    print(f"\n{'='*80}")
    print(f"Testing KB {i}: {config['name']}")
    print(f"Chunking Strategy: {config['chunking_strategy']}")
    print(f"{'='*80}\n")
    
    time.sleep(5)
    
    response = bedrock_agent_runtime_client.retrieve_and_generate(
        input={"text": query},
        retrieveAndGenerateConfiguration={
            "type": "KNOWLEDGE_BASE",
            "knowledgeBaseConfiguration": {
                'knowledgeBaseId': kb_id,
                "modelArn": f"arn:aws:bedrock:{region}::foundation-model/{foundation_model}",
                "retrievalConfiguration": {
                    "vectorSearchConfiguration": {
                        "numberOfResults": 5
                    }
                }
            }
        }
    )
    
    print("Response:")
    print(response['output']['text'])
    
    citations = response['citations'][0]['retrievedReferences']
    print(f"\nNumber of citations: {len(citations)}")
    print("="*80)

## 6. OpenSearch Serverless Collection 확인

모든 KB가 동일한 OSS Collection을 사용하지만 서로 다른 인덱스를 가지고 있는지 확인합니다.

In [ ]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

# OSS 클라이언트 생성
credentials = boto3.Session().get_credentials()
awsauth = AWSV4SignerAuth(credentials, region, 'aoss')

oss_client = OpenSearch(
    hosts=[{'host': oss_host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)

# 모든 인덱스 조회
indices = oss_client.cat.indices(format='json')

print("\nIndices in the shared OSS Collection:")
print("="*80)
for idx in indices:
    print(f"Index: {idx['index']}")
    print(f"  Docs Count: {idx['docs.count']}")
    print(f"  Store Size: {idx['store.size']}")
    print(f"  Status: {idx['health']}")
    print()

print(f"Total Indices: {len(indices)}")
print("="*80)

## 7. 정리 (선택사항)

테스트가 완료되면 리소스를 정리합니다.

In [ ]:
# 주석을 해제하여 리소스 삭제
# for i, kb in enumerate(created_kbs, start=1):
#     print(f"Deleting KB {i}...")
#     # 마지막 KB만 S3 버킷과 OSS Collection 삭제
#     delete_s3 = (i == len(created_kbs))
#     kb.delete_kb(delete_s3_bucket=delete_s3, delete_iam_roles_and_policies=True)
#     print(f"KB {i} deleted\n")

## 요약

이 노트북에서는:

1. **하나의 OpenSearch Serverless Collection 생성**
2. **여러 Knowledge Base 생성** (각각 다른 인덱스 사용)
3. **각 KB에 서로 다른 청킹 전략 적용**
4. **모든 KB가 동일한 OSS Collection 공유**

### 장점:
- OSS Collection 비용 절감
- 각 KB는 독립적으로 관리
- 서로 다른 청킹 전략 테스트 가능

### 주의사항:
- 각 KB는 서로 다른 벡터 인덱스를 사용하므로 데이터는 격리됨
- OSS Collection의 용량 제한을 고려해야 함
- 첫 번째 KB 삭제 시 OSS Collection도 함께 삭제되므로 주의